In [2]:
import pandas as pd
import numpy as np
import scipy.stats

gams_python_path="/Library/Frameworks/GAMS.framwork/Resources/apifiles/Python/gams"

In [81]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')
output=pd.read_csv('/Users/hannahkamen/Downloads/y_rpt.csv')
cps=pd.read_stata('/Users/hannahkamen/Downloads/acs5yr_0610.dta')



In [198]:
len(cps)

15057480

In [82]:
lookup=pd.read_excel('/Users/hannahkamen/Downloads/windc_cps_industry_lookup.xlsx')
lookup_final=pd.read_excel('/Users/hannahkamen/Downloads/dropped_merge.xlsx')


lookup['NAICS']=lookup['NAICS'].astype(str).str.strip()
lookup_final['NAICS']=lookup_final['NAICS'].astype(str).str.strip()
lookup_final['cps_code']=lookup_final['cps_code'].astype(str).str.strip()

In [136]:
###read in model industry pairings in final data
gams_dta=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0_d2.csv')


In [84]:
####gams labor endowment data need to check sums with sector demand
gams_dta_le0=pd.read_csv('/Users/hannahkamen/Downloads/windc_le0_d.csv')

In [85]:
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/state_lookup.xlsx')

In [86]:
state_lookup['state']=state_lookup['state'].str.strip()
state_lookup['abbrev']=state_lookup['abbrev'].str.strip()

In [87]:
###read in dropped 

In [88]:
cps_codes=pd.DataFrame(data={'cps_code':cps['indnaics'].str.strip().unique()})
windc_codes=pd.DataFrame(data={'windc_code':model['IOCode'].str.strip().unique()})

In [89]:
####match on first three symbols
cps_codes['cps_code3']=cps_codes['cps_code'].str[0:3]
lookup['NAICS3']=lookup['NAICS'].str[0:3]

merge3=lookup.merge(cps_codes, left_on='NAICS3',right_on='cps_code3',how='inner',indicator=True)

In [90]:
###append manual lookup to fill gaps
merge_f=merge3[['NAICS','cps_code']].append(lookup_final)

In [91]:
test=cps_codes.merge(merge_f,on='cps_code',how='outer',indicator=True)

In [92]:
cps_m=cps.merge(merge_f, left_on='indnaics', right_on='cps_code',how='left').merge(state_lookup, left_on='statefip',right_on='state', how='left')




In [93]:
cps_m=cps_m.rename(columns={'abbrev':'state_residence'})

In [94]:
cps_m=cps_m.merge(state_lookup, left_on='pwstate2', right_on='state',how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_work'})

In [95]:
cps_m['educ_new']=np.where(cps_m['educ'].isin(['2 years of college','4 years of college','5+ years of college']),'skl','unskl')
                           
                           

In [96]:
# cps_m['hh']=np.where(cps_m['incwage']<=25000,'hh1','')
# cps_m['hh']=np.where(((cps_m['incwage']>25000) & (cps_m['incwage']<50000)),'hh2',cps_m['hh'])
# cps_m['hh']=np.where(((cps_m['incwage']>=50000) & (cps_m['incwage']<75000)),'hh3',cps_m['hh'])
# cps_m['hh']=np.where(((cps_m['incwage']>=75000) & (cps_m['incwage']<150000)),'hh4',cps_m['hh'])
# cps_m['hh']=np.where((cps_m['incwage']>=150000),'hh5',cps_m['hh'])
 

In [97]:
cps_m['hh']=np.where(cps_m['inctot']<=25000,'hh1','')
cps_m['hh']=np.where(((cps_m['inctot']>25000) & (cps_m['inctot']<50000)),'hh2',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=50000) & (cps_m['inctot']<75000)),'hh3',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=75000) & (cps_m['inctot']<150000)),'hh4',cps_m['hh'])
cps_m['hh']=np.where((cps_m['inctot']>=150000),'hh5',cps_m['hh'])

#### get shares for consumer side, le0(r,q,h,sk)

In [98]:
gams_rqhs0=cps_m.reset_index()[['state_residence','state_work','hh','educ_new','incwage']].groupby(['state_residence','state_work','educ_new','hh'],as_index=False).agg({'incwage':sum})



In [99]:
gams_rqhs=gams_rqhs0.rename(columns={'state_residence':'r','state_work':'q','hh':'h','educ_new':'sk'})
gams_rqhs=gams_rqhs[['r','q','h','sk','incwage']]

In [100]:
midwest=['IA','OH','WI','NE','IL','MI','SD','ND','MN','MO','IN','KS']
south=['FL','MD','TN','WV','OK','KY','NC','VA','DE','GA','MS','TX','AL','LA','AR','SC','DC']
west=['AK','AZ','NM','HI','CA','WA','NV','OR','ID','UT','MT','WY','CO']
northeast=['VT','NH','CT','ME','MA','NY','NJ','PA','RI']

In [101]:
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(midwest),'midwest','')
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(south),'south',gams_rqhs['census_region_r'])
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(west),'west',gams_rqhs['census_region_r'])
gams_rqhs['census_region_r']=np.where(gams_rqhs['r'].isin(northeast),'northeast',gams_rqhs['census_region_r'])


gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(midwest),'midwest','')
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(south),'south',gams_rqhs['census_region_q'])
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(west),'west',gams_rqhs['census_region_q'])
gams_rqhs['census_region_q']=np.where(gams_rqhs['q'].isin(northeast),'northeast',gams_rqhs['census_region_q'])

In [102]:
gams_rqhs_lm=gams_rqhs[(gams_rqhs['census_region_r']==gams_rqhs['census_region_q'])]

In [103]:
gams_rqhs_lm_gr=gams_rqhs_lm.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
gams_rqhs_lm_gr=gams_rqhs_lm_gr.rename(columns={'incwage':'incwage_total'})
gams_rqhs_lm1=gams_rqhs_lm.merge(gams_rqhs_lm_gr, on=['r','q','h'],how='left')
gams_rqhs_lm1['skl_shr']=gams_rqhs_lm1['incwage']/gams_rqhs_lm1['incwage_total']



In [104]:
gams_rqhs_lm1[(gams_rqhs_lm1['r']=='AK') &(gams_rqhs_lm1['q']=='AK')&(gams_rqhs_lm1['h']=='hh3')]

,r,q,h,sk,incwage,census_region_r,census_region_q,incwage_total,skl_shr
2,AK,AK,hh3,skl,120977222.0,west,west,240192878.0,0.503667
7,AK,AK,hh3,unskl,119215656.0,west,west,240192878.0,0.496333


In [31]:
gams_dta_le0.head()

,file,flx,int,scn,esubw_oth,sector,seniority,short,value
0,sm_test,IA,IA,hh1,skl,0.273961,NaN,NaN,NaN
1,sm_test,IA,IA,hh1,unskl,0.765274,NaN,NaN,NaN
2,sm_test,IA,IA,hh2,skl,1.827777,NaN,NaN,NaN
3,sm_test,IA,IA,hh2,unskl,2.620903,NaN,NaN,NaN
4,sm_test,IA,IA,hh3,skl,4.429970,NaN,NaN,NaN


In [69]:
len(gams_dta_le0)

6830

In [70]:
len(merge_gams)

6830

#### Deal with missing combinations in CPS data for r,q,h,sk

In [105]:
##prepare gams table for merge
gams_dta_le0=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk',' sector':'pop'})[['r','q','h','sk','pop']]
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(gams_dta_le0,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tage to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']

###if region sector combination is not missing, use census region values, otherwise keep original share value
final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])




###limit dataframe

final_shrs0=final_shrs0[['r','q','h','sk','skill_shr','combo','pop','census_share']]

#chksum to see that all proportions add to 1
check_sum0=final_shrs0.groupby(['r','q','h'],as_index=False).agg({'skill_shr':sum})
check_sum0=check_sum0[check_sum0['skill_shr']!=1]


#chksum to see that no zeros exist
check_sum10=final_shrs0.groupby(['r','q','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]

zeros=check_sum10['r']+"_"+check_sum10['q']+"_"+check_sum10['h']


#replace 0s with census values

final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])



#check zeros agains
check_sum10=final_shrs0.groupby(['r','q','h','sk'],as_index=False).agg({'skill_shr':sum})
check_sum10=check_sum10[check_sum10['skill_shr']==0]

final_shrs0_f=final_shrs0[['r','q','h','sk','skill_shr']]

In [110]:
final_shrs0[final_shrs0['combo'].isin(missing_combos0)]['pop'].sum()

469.6733563736173

In [111]:
469.6733563736173/9215

0.050968351207120705

In [107]:
len(final_shrs0)

6830

In [106]:
missing_combos0

55      CO_HI_hh1
75      HI_OR_hh1
77      HI_UT_hh1
125     NV_CO_hh1
145     OR_NM_hh1
          ...    
6825    ND_MO_hh5
6826    ND_IN_hh5
6827    ND_IN_hh5
6828    ND_KS_hh5
6829    ND_KS_hh5
Length: 1970, dtype: object

In [108]:
1970/6830

0.2884333821376281

In [55]:
[x[3:5] for x in missing_combos0]

['OR',
 'UT',
 'WY',
 'HI',
 'UT',
 'CO',
 'HI',
 'NM',
 'WY',
 'NM',
 'NM',
 'HI',
 'NV',
 'NV',
 'OR',
 'UT',
 'UT',
 'MT',
 'WY',
 'WY',
 'CO',
 'CO',
 'AK',
 'HI',
 'OR',
 'MT',
 'AK',
 'HI',
 'HI',
 'WA',
 'OR',
 'OR',
 'ID',
 'ID',
 'MT',
 'MT',
 'WY',
 'AK',
 'AZ',
 'NM',
 'NM',
 'ID',
 'MT',
 'MT',
 'MT',
 'WY',
 'WY',
 'WY',
 'WY',
 'NM',
 'NM',
 'CO',
 'AK',
 'NM',
 'HI',
 'OR',
 'AK',
 'NM',
 'NM',
 'HI',
 'HI',
 'WA',
 'AK',
 'AZ',
 'AZ',
 'NM',
 'NM',
 'HI',
 'WA',
 'WA',
 'OR',
 'OR',
 'OR',
 'MT',
 'MT',
 'CO',
 'OR',
 'UT',
 'WA',
 'WY',
 'AK',
 'ID',
 'WA',
 'AZ',
 'HI',
 'HI',
 'UT',
 'AZ',
 'NM',
 'NM',
 'HI',
 'HI',
 'CA',
 'CA',
 'WA',
 'WA',
 'NV',
 'NV',
 'OR',
 'OR',
 'ID',
 'ID',
 'UT',
 'UT',
 'MT',
 'MT',
 'WY',
 'WY',
 'CO',
 'CO',
 'AK',
 'AK',
 'HI',
 'HI',
 'OR',
 'AK',
 'HI',
 'HI',
 'OR',
 'ID',
 'ID',
 'MT',
 'MT',
 'AK',
 'AK',
 'AZ',
 'AZ',
 'NM',
 'NM',
 'NV',
 'ID',
 'ID',
 'MT',
 'MT',
 'WY',
 'WY',
 'ID',
 'AZ',
 'WY',
 'WY',
 'CO',
 'NM',
 'NM',

In [75]:
final_shrs['pop'].sum()

9215.754554865574

In [74]:
gams_dta[' esubw_oth'].sum()

9215.754554865574

In [46]:
final_shrs0.head()

,r,q,h,sk,skill_shr,combo,pop,census_share
0,AK,AK,hh1,skl,0.210555,AK_AK_hh1,0.038152,0.264425
1,AK,AK,hh1,unskl,0.789445,AK_AK_hh1,0.143047,0.735575
2,AK,AZ,hh1,skl,0.264425,AK_AZ_hh1,0.002001,0.264425
3,AK,AZ,hh1,unskl,0.735575,AK_AZ_hh1,0.005567,0.735575
4,AK,CA,hh1,skl,0.491478,AK_CA_hh1,0.003731,0.264425


In [37]:
##check length should be 6830
len(final_shrs0)

6830

In [38]:
#show dataset if any total shares are not 1
check_sum0

,r,q,h,skill_shr


In [39]:
#show dataset if any shares are 0
check_sum10

,r,q,h,sk,skill_shr


#### get shares for producer side  ld0(r,s,sk)


In [200]:
len(cps_m)

17612716

In [40]:
# group on sector
gams_rssk0=cps_m.reset_index()[['state_work','NAICS','educ_new','incwage']].groupby(['state_work','NAICS','educ_new'],as_index=False).agg({'incwage':sum})
#get rid of puerto rico and "bad" sectors



In [43]:
##rename to gams indices
gams_rssk=gams_rssk0.rename(columns={'state_work':'r','NAICS':'s','educ_new':'sk'})
gams_rssk=gams_rssk[['r','s','sk','incwage']]

In [44]:
model['gams.Commodities/Industries'].unique()

array(['agr', 'fof', 'oil', 'min', 'smn', 'uti', 'con', 'wpd', 'nmp',
       'pmt', 'fmt', 'mch', 'cep', 'eec', 'mot', 'ote', 'fpd', 'mmf',
       'fbp', 'tex', 'alt', 'ppd', 'pri', 'pet', 'che', 'pla', 'wht',
       'mvt', 'fbt', 'gmt', 'ott', 'air', 'trn', 'wtt', 'trk', 'grd',
       'pip', 'otr', 'wrh', 'pub', 'mov', 'brd', 'dat', 'bnk', 'sec',
       'ins', 'fin', 'hou', 'ore', 'rnt', 'leg', 'com', 'tsv', 'man',
       'adm', 'wst', 'edu', 'amb', 'hos', 'nrs', 'soc', 'art', 'rec',
       'amd', 'res', 'osv', 'fdd', 'fnd', 'fen', 'slg', 'sle', 'Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply'],
      dtype=object)

In [45]:
model['gams.Commodities/Industries'].unique()

non_s=['Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply']

In [47]:
model_gr=model.groupby(['IOCode','gams.IOCode'],as_index=False).sum()
model_gr=model_gr[['IOCode','gams.IOCode']]

In [48]:
gams_rssk_m=gams_rssk.merge(model_gr, left_on='s',right_on='IOCode',how='left')
gams_rssk_m=gams_rssk_m[gams_rssk_m['gams.IOCode']!='use']
gams_rssk_m=gams_rssk_m[gams_rssk_m['r']!='PR']
del gams_rssk_m['s']
del gams_rssk_m['IOCode']
gams_rssk_m=gams_rssk_m.rename(columns={'gams.IOCode':'s'})
gams_rssk_m=gams_rssk_m[['r','s','sk','incwage']]

#### figure out which combinations are not in CPS but in GAMS tables

In [137]:
##prepare gams table for merge
gams_dta=gams_dta.rename(columns={' flx':'r',' int':'s',' scn':'sk',' esubw_oth':'pop'})[['r','s','sk','pop']]
##outer merge CPS data with gams table with indicator
merge_sector=gams_rssk_m.merge(gams_dta,on=['r','s','sk'],how='outer',indicator=True)

###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_sector['census_region']=np.where(merge_sector['r'].isin(midwest),'midwest','')
merge_sector['census_region']=np.where(merge_sector['r'].isin(south),'south',merge_sector['census_region'])
merge_sector['census_region']=np.where(merge_sector['r'].isin(west),'west',merge_sector['census_region'])
merge_sector['census_region']=np.where(merge_sector['r'].isin(northeast),'northeast',merge_sector['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot=merge_sector.groupby(['census_region','s'],as_index=False).agg({'incwage':sum})
census_avg_tot=census_avg_tot.rename(columns={'incwage':'incwage_tot'})
census_avg=merge_sector.groupby(['census_region','s','sk'],as_index=False).agg({'incwage':sum})

census_avg=census_avg.merge(census_avg_tot,on=['census_region','s'],how='inner')
census_avg['census_share']=census_avg['incwage']/census_avg['incwage_tot']
del census_avg['incwage']
del census_avg['incwage_tot']

# merge census averages onto original merged dataframe
merge_sector_cs=merge_sector.merge(census_avg,on=['census_region','s','sk'],how='inner')

##keep only inner values and values missing from CPS

final=merge_sector_cs[merge_sector_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot=final.groupby(['r','s'],as_index=False).agg({'incwage':sum})
final_tot=final_tot.rename(columns={'incwage':'incwage_tot'})


final_shrs=final.merge(final_tot,on=['r','s'],how='inner')
final_shrs['skill_shr']=final_shrs['incwage']/final_shrs['incwage_tot']
del final_shrs['incwage']
del final_shrs['incwage_tot']

#create tage to define missing combinations in final dataset

final_shrs['combo']=final_shrs['r']+"_"+final_shrs['s']

## get list of missing combionations using _merge indicator var

missing_combos=final_shrs[final_shrs['_merge']=='right_only']['r']+"_"+final_shrs[final_shrs['_merge']=='right_only']['s']

###if region sector combination is not missing, use census region values, otherwise keep original share value
final_shrs['skill_shr']=np.where(final_shrs['combo'].isin(missing_combos),final_shrs['census_share'],final_shrs['skill_shr'])
final_shrs['skill_shr']=np.where(final_shrs['combo']=='AK_alt',final_shrs['census_share'],final_shrs['skill_shr'])

###limit dataframe

final_shrs=final_shrs[['r','s','sk','skill_shr','pop','census_share']]


#chksum to see that all proportions add to 1
check_sum=final_shrs.groupby(['r','s'],as_index=False).agg({'skill_shr':sum})
check_sum[check_sum['skill_shr']!=1]

#chksum to see t no zeros exist
check_sum1=final_shrs.groupby(['r','s','sk'],as_index=False).agg({'skill_shr':sum})
check_sum1[check_sum1['skill_shr']==0]

final_shrs_f=final_shrs[['r','s','sk','skill_shr']]



In [78]:
final_shrs0_f.head()

,r,q,h,sk,skill_shr
0,AK,AK,hh1,skl,0.210555
1,AK,AK,hh1,unskl,0.789445
2,AK,AZ,hh1,skl,0.264425
3,AK,AZ,hh1,unskl,0.735575
4,AK,CA,hh1,skl,0.491478


In [ ]:
#population that is in gams based on total labor endowment in the working region, sector demand is the same ratio for every working region


In [115]:
le0_ssk=final_shrs0.groupby(['q','sk'],as_index=False).agg({'pop':sum})

In [138]:
gams_dta

,r,s,sk,pop
0,IA,ppd,skl,0.226799
1,IA,ppd,unskl,0.099038
2,IA,res,skl,1.309129
3,IA,res,unskl,0.571665
4,IA,com,skl,0.779061
...,...,...,...,...
7197,CO,otr,unskl,0.229072
7198,CO,min,skl,0.376623
7199,CO,min,unskl,0.108725
7200,CO,oil,skl,1.636361


In [123]:
len(set([x[0:2] for x in missing_combos]))

19

In [124]:
len(set([x[3:5] for x in missing_combos0]))

51

In [127]:
le0_ssk

,q,sk,pop
0,AK,skl,22.030937
1,AK,unskl,13.762905
2,AL,skl,72.570488
3,AL,unskl,29.145400
4,AR,skl,36.088835
...,...,...,...
97,WI,unskl,42.336603
98,WV,skl,21.141851
99,WV,unskl,11.821189
100,WY,skl,9.240708


In [133]:
gams_dta

,r,s,sk,pop
0,IA,ppd,skl,0.249336
1,IA,ppd,unskl,0.076501
2,IA,res,skl,1.439216
3,IA,res,unskl,0.441577
4,IA,com,skl,0.856475
...,...,...,...,...
7197,CO,otr,unskl,0.190513
7198,CO,min,skl,0.394924
7199,CO,min,unskl,0.090423
7200,CO,oil,skl,1.715876


In [141]:
ld0_gr_r_s=gams_dta.groupby(['r','s'],as_index=False).agg({'pop':sum})
ld0_gr_r_s=ld0_gr_r_s.rename(columns={'pop':'tot_pop_r_s'})

In [160]:
le0_ssk[(le0_ssk['q']=='WI') & (le0_ssk['sk']=='unskl')]

,q,sk,pop
97,WI,unskl,42.336603


In [177]:
test_wi

,r,s,sk,skill_shr
5515,WI,agr,unskl,0.767903
5539,WI,fof,unskl,0.562789
5561,WI,oil,unskl,0.567303
5585,WI,min,unskl,0.803332
5609,WI,smn,unskl,0.739512
...,...,...,...,...
7097,WI,fnd,unskl,0.377255
7121,WI,sle,unskl,0.377255
7145,WI,slg,unskl,0.377255
7169,WI,hou,unskl,0.502843


In [180]:
gams_dta[gams_dta['r']=='WI']['pop'].sum()

147.41713407500293

In [193]:
test_wi=final_shrs[(final_shrs['r']=='WI') & (final_shrs['sk']=='unskl')][['r','s','sk','skill_shr']]

In [194]:
gams_dta_sumoversk=gams_dta.groupby(['r','s'],as_index=False).agg({'pop':sum})
gams_dta_sumoversk=gams_dta_sumoversk.rename(columns={'pop':'total_labor_in_WI_by_sector'})

In [195]:
test_wi=test_wi.merge(gams_dta_sumoversk,on=['r','s'])
test_wi['labor_demand_based_on_cps']=test_wi['skill_shr']*test_wi['total_labor_in_WI_by_sector']

In [197]:
test_wi['labor_demand_based_on_cps'].sum()

70.30285649587054

In [147]:
#del final_shrs['pop']
#merge total workers in that region by skill type
final_shrs_m=final_shrs.merge(le0_ssk,left_on=['r','sk'], right_on=['q','sk'])
final_shrs_m=final_shrs_m.rename(columns={'pop':'tot_pop_r_sk'})
#merge total workers demanded in that region by a given sector

final_shrs_m=final_shrs_m.merge(ld0_gr_r_s,on=['r','s'])

In [149]:
final_shrs_m['labor_demand_by_r_s_sk']=final_shrs_m['skill_shr']*final_shrs_m['tot_pop_r_s']

In [154]:
realized_tot_pop_r_sk=final_shrs_m.groupby(['r','sk'],as_index=False).agg({'labor_demand_by_r_s_sk':sum})

realized_tot_pop_r_sk=realized_tot_pop_r_sk.rename(columns={'labor_demand_by_r_s_sk':'realized_tot_pop_r_sk'})

final_shrs_m=final_shrs_m.merge(realized_tot_pop_r_sk,on=['r','sk'])

In [159]:
final_shrs_m

,r,s,sk,skill_shr,census_share,q,tot_pop_r_sk,tot_pop_r_s,labor_demand_by_r_s_sk,realized_tot_pop_r_sk
0,AK,agr,skl,0.488312,0.277187,AK,22.030937,0.013264,0.006477,18.596776
1,AK,fof,skl,0.372541,0.417839,AK,22.030937,0.082341,0.030675,18.596776
2,AK,oil,skl,0.640693,0.535124,AK,22.030937,1.087682,0.696871,18.596776
3,AK,min,skl,0.265812,0.306666,AK,22.030937,0.430119,0.114331,18.596776
4,AK,smn,skl,0.346153,0.348822,AK,22.030937,0.791123,0.273850,18.596776
...,...,...,...,...,...,...,...,...,...,...
7197,WI,fnd,unskl,0.377255,0.413121,WI,42.336603,3.684775,1.390099,70.302856
7198,WI,sle,unskl,0.377255,0.413121,WI,42.336603,0.460510,0.173730,70.302856
7199,WI,slg,unskl,0.377255,0.413121,WI,42.336603,4.316034,1.628244,70.302856
7200,WI,hou,unskl,0.502843,0.431614,WI,42.336603,0.847804,0.426312,70.302856


In [157]:
len(final_shrs_m)

7202

In [634]:
#final_shrs0_popm_gr.to_csv('/Users/hannahkamen/Downloads/le0_s_shr.csv',index=False)

final_shrs0.to_csv('/Users/hannahkamen/Downloads/le0_shr.csv',index=False)
#final_shrs.to_csv('/Users/hannahkamen/Downloads/ld0_shr.csv',index=False)

In [ ]:
final_shrs.groupby(['r','sk',])

In [582]:
check_sum[check_sum['skill_shr']!=1]

,r,s,skill_shr


In [583]:
check_sum1[check_sum1['skill_shr']==1]

,r,s,sk,skill_shr


In [584]:
len(gams_dta)

7202

In [585]:
len(final_shrs)

7202

In [421]:
# in_table=gams_rssk_m['gams.IOCode'].unique()
# in_model_dta=gams_dta[' int'].unique()
# [x for x in in_table if x not in in_model_dta]
# test1=gams_rssk['s'].unique()
# test2=model[~model['gams.Commodities/Industries'].isin(non_s)]['gams.Commodities/Industries'].unique()
# test3=[x for x in test2 if x not in test1 ]
# print(len(midwest))
# print(len(south))
# print(len(west))
# print(len(northeast))